In [3]:
import numpy as np
estimated_tardiness = np.full(4, 2)

print(estimated_tardiness)

A = estimated_tardiness[2]
print(A)

[2 2 2 2]
2


In [1]:
import pandas as pd
import plotly.express as px

# Sample DataFrame
data = {
    'Job': [0, 0, 0, 1, 1, 1, 2, 2, 3, 3, 4, 4, 5],
    'Ope': [0, 1, 2, 0, 1, 2, 0, 1, 0, 1, 0, 1, 0],
    'MC': [1, 6, 4, 1, 7, 4, 1, 5, 5, 1, 1, 5, 1],
    'Start': [68, 72, 75, 1, 9, 12, 9, 25, 83, 98, 86, 102, 79],
    'End': [72, 75, 99, 9, 12, 36, 12, 29, 86, 102, 89, 106, 82]
}

df = pd.DataFrame(data)

# Plot using Plotly Express
fig = px.timeline(df, x_start='Start', x_end='End', y='MC', color='Job', text='Job',
                  labels={'MC': 'Machine Center', 'Job': 'Job ID'},
                  title='Machine Center Utilization Gantt Chart')
fig.update_yaxes(categoryorder='total ascending')
fig.update_traces(textposition='inside')
fig.show()


In [1]:
import numpy as np
import time
import random
import copy
from env_action.metaheu import decoding

def get_cumulative_probability(k, operation, job, n_MC_ji, MC_ji):
    # Identify the index of k in MC_ji
    index_k = MC_ji[job][operation].index(k)
    
    # Determine the segment range
    segment_size = 1 / n_MC_ji[job][operation]
    lower_bound = index_k * segment_size
    upper_bound = (index_k + 1) * segment_size
    
    # Generate a random number within the segment range
    random_number = random.uniform(lower_bound, upper_bound)
    
    return random_number

def generate_neighborhood(solution, neighborhood_size, chromosome_len):
    current_OA = solution[0]
    current_MS = solution[1]
    neighborhood = set()
    
    while len(neighborhood) < neighborhood_size:
        # Randomly choose two distinct positions for insertion
        i, j        = random.sample(range(chromosome_len), 2)
        neighbor_OA = copy.deepcopy(current_OA)
        element_OA  = neighbor_OA.pop(i)
        neighbor_OA.insert(j, element_OA)

        i, j        = random.sample(range(chromosome_len), 2)
        neighbor_MS = current_MS[:]
        element_MS  = neighbor_MS.pop(i)
        neighbor_MS.insert(j, element_MS)

        neighbor = (neighbor_OA, neighbor_MS)

        neighborhood.add(tuple(neighbor))
        
        # Randomly choose two adjacent positions for swapping
        k = random.randint(0, chromosome_len - 2)
        neighbor_OA = current_OA[:]
        neighbor_OA[k], neighbor_OA[k + 1] = neighbor_OA[k + 1], neighbor_OA[k]

        k = random.randint(0, chromosome_len - 2)
        neighbor_MS = current_MS[:]
        neighbor_MS[k], neighbor_MS[k + 1] = neighbor_MS[k + 1], neighbor_MS[k]

        neighbor = (neighbor_OA, neighbor_MS)
        neighborhood.add(tuple(neighbor))
        
        # Crossover
        crossover_point = random.randint(1, chromosome_len - 1)
        neighbor_OA = current_OA[crossover_point:] + current_OA[:crossover_point]

        crossover_point = random.randint(1, chromosome_len - 1)
        neighbor_MS = current_MS[crossover_point:] + current_MS[:crossover_point]

        neighbor = (neighbor_OA, neighbor_MS)
        neighborhood.add(tuple(neighbor))
    
    return [list(neighbor) for neighbor in neighborhood]


def TabuSearch (S_k, S_j, JSet, J, I, K, 
                p_ijk, h_ijk, d_j, n_j, n_ops_left_j, 
                MC_ji, n_MC_ji, OperationPool, 
                X_ijk, S_ij, C_ij, t, maxtime):
    
    max_Generation      = 500
    max_No_improve      = 10
    start_time          = time.time()
    GBest               = float('inf')
    tabu_list_size      = 10
    neighborhood_size   = 5

    """Initialize"""
    # Take the machine for each operation
    X_ij = np.argmax(X_ijk, axis=2)
    # Flat
    S_ij[S_ij == -999] = np.inf
    S_ij_flat = S_ij.flatten()
    X_ij_flat = X_ij.flatten()

    print(S_ij_flat)

    indices = np.indices((I, J))
    ope = np.stack(indices, axis=-1)

    # Sort
    sorted_indices = np.argsort(S_ij_flat)

    sorted_S_ij = S_ij_flat[sorted_indices]
    sorted_X_ij = X_ij_flat[sorted_indices]
    sorted_ope  = ope [sorted_indices]

    running_mask = sorted_S_ij < t
    sorted_S_ij = sorted_S_ij[running_mask]
    sorted_X_ij = sorted_X_ij[running_mask]
    sorted_ope  = sorted_ope [running_mask]

    OA             = np.array([ope[1] for ope in sorted_ope])
    chromosome_len = len(OA)
    MS             = np.zeros((chromosome_len))
    for ope in len(OA):
        i, j = sorted_ope[ope]
        k    = sorted_X_ij[ope]
        MS[ope] = get_cumulative_probability[k, i, j, n_MC_ji, MC_ji]

    if JA:
        for new_job in JA:
            chromosome_len += n_j[newjob]
            if urgent:
                
            else:
                

    current_solution    = (OA, MS)
    GSol                = copy.deepcopy(current_solution)
    GBest, X, S, C, Cj  = decoding(S_k, S_j, GSol[0], GSol[1], chromosome_len, 
                                   n_MC_ji, MC_ji, I, J, K, JSet, p_ijk, d_j, n_j, n_ops_left_j)

    tabu_list    = []
    generation   = 0
    no_improve   = 0
    elapsed_time = 0
    while generation < max_Generation and no_improve < max_No_improve and elapsed_time < maxtime:
        # Generate the neighborhood of the current solution
        neighborhood = generate_neighborhood(current_solution, neighborhood_size, chromosome_len)
        
        # Find the best non-tabu solution in the neighborhood
        best_neighbor = None
        best_neighbor_objective = float('inf')
        for neighbor in neighborhood:
            if neighbor not in tabu_list:
                neighbor_objective, X, S, C, Cj = decoding(S_k, S_j, neighbor[0], neighbor[1], chromosome_len, 
                                                           n_MC_ji, MC_ji, I, J, K, JSet, p_ijk, d_j, n_j, n_ops_left_j)
                if neighbor_objective < best_neighbor_objective:
                    best_neighbor               = copy.deepcopy(neighbor)
                    best_neighbor_objective     = copy.deepcopy(neighbor_objective)
        
        # Update the current solution and the best solution
        current_solution = best_neighbor
        if best_neighbor_objective < GBest:
            GSol  = copy.deepcopy(best_neighbor)
            GBest = copy.deepcopy(best_neighbor_objective)
            no_improve = 0
        else:
            no_improve += 1
        
        # Add the current solution to the tabu list
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_list_size:
            tabu_list.pop(0)

        generation += 1
        elapsed_time = time.time() - start_time
        

    GBest, X_ijk, S_ij, C_ij, C_j = decoding(S_k, S_j, GSol[0], GSol[1], chromosome_len, 
                                             n_MC_ji, MC_ji, I, J, K, JSet, p_ijk, d_j, n_j, n_ops_left_j)

    return GBest, X_ijk, S_ij, C_ij, C_j









CUDA is not available. Using CPU.


In [14]:
import random

def generate_neighborhood(solution, neighborhood_size):
    neighborhood = set()
    
    while len(neighborhood) < neighborhood_size:
        # Randomly choose two distinct positions for insertion
        i, j = random.sample(range(len(solution)), 2)
        neighbor = solution[:]
        element = neighbor.pop(i)
        neighbor.insert(j, element)
        neighborhood.add(tuple(neighbor))
        
        # Randomly choose two adjacent positions for swapping
        k = random.randint(0, len(solution) - 2)
        neighbor = solution[:]
        neighbor[k], neighbor[k + 1] = neighbor[k + 1], neighbor[k]
        neighborhood.add(tuple(neighbor))
        
        # Randomly shift the solution
        shift_amount = random.randint(1, len(solution) - 1)
        neighbor = solution[shift_amount:] + solution[:shift_amount]
        print("-----------", neighbor)
        neighborhood.add(tuple(neighbor))
    
    return [list(neighbor) for neighbor in neighborhood]

# Example usage
current_solution = [1, 2, 3, 4, 5, 6, 7, 8, 9 , 10]
neighborhood = generate_neighborhood(current_solution, 5)
for neighbor in neighborhood:
    print(neighbor)


----------- [8, 9, 10, 1, 2, 3, 4, 5, 6, 7]
----------- [8, 9, 10, 1, 2, 3, 4, 5, 6, 7]
[8, 9, 10, 1, 2, 3, 4, 5, 6, 7]
[1, 2, 4, 3, 5, 6, 7, 8, 9, 10]
[1, 2, 3, 4, 6, 5, 7, 8, 9, 10]
[2, 3, 1, 4, 5, 6, 7, 8, 9, 10]
[1, 2, 4, 5, 6, 7, 8, 9, 10, 3]
